**Introduction**[[0\]](https://www.transitwiki.org/TransitWiki/index.php/Four-step_travel_model)

The four-step travel model is a ubiquitous framework for determining transportation forecasts that goes back to the 1950s. It was one of the first travel demand models that sought to link land use and behavior to inform transportation planning[[1\]](https://www.transitwiki.org/TransitWiki/index.php/Four-step_travel_model#cite_note-mcnally-2). Originally applied in the highway planning context, the model was expanded in the 1970s and 1980s to include multimodal trips and improved modelling techniques[[1\]](https://www.transitwiki.org/TransitWiki/index.php/Four-step_travel_model#cite_note-mcnally-2).

**The Four Steps**

The four steps are described as follows:[[2\]](https://www.transitwiki.org/TransitWiki/index.php/Four-step_travel_model#cite_note-3)

**Trip Generation**

Trip generation determines the frequency of origins or destinations of trips in each zone by trip purpose, as a function of land use, household demographics, and other socioeconomic factors.

**Trip Distribution**

Trip distribution matches origins with destinations, often using a gravity model– a calculation that takes into account the relative activity at the origin and destination as well as the travel cost to go between them[[3\]](https://www.transitwiki.org/TransitWiki/index.php/Four-step_travel_model#cite_note-4).

**Mode Choice**

Mode choice computes the proportion of trips between each origin and destination that use a particular transportation mode. (This modal model may be of the logit form, developed by Nobel Prize winner Daniel McFadden.)

**Route Assignment**

Route assignment allocates trips between an origin and destination by a particular mode to a route. Often (for highway route assignment) Wardrop's principle of user equilibrium is applied (equivalent to a Nash equilibrium), wherein each driver (or group) chooses the shortest (travel time) path, subject to every other driver doing the same. The difficulty is that travel times are a function of demand, while demand is a function of travel time, the so-called bi-level problem. Another approach is to use the Stackelberg competition model, where users ("followers") respond to the actions of a "leader", in this case for example a traffic manager. This leader anticipates on the response of the followers.


# 1 Data preparation and trip generation

## 1.1 Import python packages

In [11]:
import os
import csv
import numpy as np
import pandas as pd
from math import acos, sin, cos

## 1.2 Defining Zone class

In [12]:
class Zone:
    def __init__(self):
        self.id = 0
        self.name = ''  # alphanumeric name of this zone, such as A1, A2,...
        self.bin_index = 0
        self.activity_nodes = ''  # activity nodes which belong to this zone

        self.centroid = ''  # centroid coordinate (x, y) based on wkt format
        self.centroid_x = ''
        self.centroid_y = ''
        self.geometry = ''  # zone boundary based on wkt format

        self.production = 0.0
        self.attraction = 0.0

## 1.3 Define functions to read files (zone.csv and od_accessibility.csv)

In [13]:
def read_zone_csv(path_zone_csv: str=None) -> list:
    """read zone.csv obtained from path4gmns"""

    number_of_zones = 0
    g_zone_list = []
    g_zone_id_to_index_dict = {}

    with open(path_zone_csv, errors='ignore') as fp:
        reader = csv.DictReader(fp)

        print("Start reading zone.csv")
        zone_index = 0
        for line in reader:

            # Create Zone objects
            if line.get('zone_id', None):

                # initialize a zone object
                zone = Zone()

                zone.id = int(line.get("zone_id")) if line.get("zone_id") else 0
                g_zone_id_to_index_dict[zone.id] = zone_index if line.get('zone_id') else None

                zone.bin_index = int(line.get('bin_index')) if line.get('bin_index') else 0
                zone.activity_nodes = line.get('activity_nodes') or ''

                zone.centroid_x = line.get('x_coord') or ''
                zone.centroid_y = line.get('y_coord') or ''
                zone.centroid = f'POINT ({zone.centroid_x} {zone.centroid_y})'
                zone.geometry = line.get('geometry') or ''

                zone.production = float(line.get('production')) if line.get('production') else 0.0
                zone.attraction = float(line.get('attraction')) if line.get('attraction') else 0.0

                zone_index += 1
                number_of_zones += 1
                g_zone_list.append(zone)

    if number_of_zones == 0:
        print("zone information is empty in zone.csv! Please check it.")
    else:
        print("Finished! The number of zones is", number_of_zones)

    return [g_zone_list, g_zone_id_to_index_dict]

path_zone_csv = "./zone_test.csv"
g_zone_list, g_zone_id_to_index_dict = read_zone_csv(path_zone_csv)


Start reading zone.csv
Finished! The number of zones is 590


In [14]:

def convert_geometry_str_to_points(geometry_str: str) -> list:
    """Convert geometry string to a list of points
    Args:
        geometry_str (str): LINESTRING (-122.28335 47.6470736,-122.28335 47.6470736)
    Return:
        point1 (list): [longitude, latitude]
		point2 (list): [longitude, latitude]
    """

    point1, point2 = [i.split(" ") for i in geometry_str.split(
        "LINESTRING ")[-1][1:-1].split(",")]
    return [point1, point2]

def calculate_distance_from_two_points(point1: list, point2: list) -> float:
	"""Calculate distance between two points on the earth surface based on longitude and latitude

	Args:
		point1 (list): [longitude, latitude] of point 1
		point2 (list): [longitude, latitude] of point 2

 	Return:
  		distance (float): unit in mile
	"""

	Equatorial_Radius = 3963.19059 # unit: mile

	p1lng = np.radians(float(point1[0]))
	p1lat = np.radians(float(point1[1]))
	p2lng = np.radians(float(point2[0]))
	p2lat = np.radians(float(point2[1]))

	distance=acos(sin(p1lat)*sin(p2lat)+cos(p1lat)*cos(p2lat)*cos(p2lng-p1lng)) * Equatorial_Radius
	return distance

def read_accessibility_csv(path_accessibility: str) -> list:
    """ Read od_accessibility.csv obtained from path4gmns"""

    # initialize variables
    g_o_zone_id_list = []
    g_o_zone_name_list = []
    g_d_zone_id_list = []
    g_d_zone_name_list = []
    g_od_distance_list = []
    g_od_geometry_list = []

    # read file and store data
    with open(path_accessibility, errors='ignore') as fp:
        reader = csv.DictReader(fp)

        print("\nStart reading od_accessibility.csv")
        for line in reader:

            # origin and destination zone ids are valid
            if all([line.get('o_zone_id'), line.get('d_zone_id')]):

                g_o_zone_id_list.append(int(line.get('o_zone_id')))
                g_o_zone_name_list.append(line.get('o_zone_name'))

                g_d_zone_id_list.append(int(line.get('d_zone_id')))
                g_d_zone_name_list.append(line.get('d_zone_name'))

                geometry_value = line.get('geometry') or ''
                g_od_geometry_list.append(geometry_value)

                try:
                    # if distance column exists
                    g_od_distance_list.append(float(line['distance']))
                except Exception:
                    # geometry value is not None or ''
                    if geometry_value:
                        point1,point2 = convert_geometry_str_to_points(geometry_value)
                        g_od_distance_list.append(calculate_distance_from_two_points(point1, point2))
                    else:
                        g_od_distance_list.append(float(999999))
                        print(f"distance between OD pair {line['o_zone_id']}-{line['d_zone_id']} is None in od_accessibility.csv! Default distance 999999 is used.")
                        print(f"geometry field is empty for OD pair {line['o_zone_id']}-{line['d_zone_id']} in od_accessibility.csv! Please check it.")

    nearest_distance = min(g_od_distance_list)
    farthest_distance = max(g_od_distance_list)
    print("Finished!")
    print("The nearest distance between OD pairs is", nearest_distance)
    print("The farthest distance between OD pairs is",farthest_distance)

    return [g_o_zone_id_list,g_d_zone_id_list,g_o_zone_name_list,g_d_zone_name_list,g_od_distance_list,g_od_geometry_list]


path_accessibility = "./accessibility.csv"
g_o_zone_id_list,g_d_zone_id_list,g_o_zone_name_list,g_d_zone_name_list,g_od_distance_list,g_od_geometry_list = read_accessibility_csv(path_accessibility)



Start reading od_accessibility.csv
Finished!
The nearest distance between OD pairs is 0.0
The farthest distance between OD pairs is 3.205418702200364


# 2 Trip Distribution

##  2.1 Parameters setting

In [15]:
# Generate gamma function coefficients for friction factors
trip_purpose_list = ['HBW','HBO','NHB']
a_list = [28507,139173,219113]
b_list = [-0.02,-1.285,-1.332]
c_list = [-0.123,-0.094,-0.1]

data = pd.DataFrame({
    'trip_purpose': trip_purpose_list,
    'a': a_list,
    'b': b_list,
    'c': c_list})

print(data)

  trip_purpose       a      b      c
0          HBW   28507 -0.020 -0.123
1          HBO  139173 -1.285 -0.094
2          NHB  219113 -1.332 -0.100


In [16]:
# Calculate travel time between zones (unit: km/h)
average_travel_speed = 30
trip_purpose_index = 0
a,b,c = [factor[trip_purpose_index] for factor in [a_list, b_list, c_list]]
print(a,b,c)

28507 -0.02 -0.123


## 2.2 Calculate friction factors between zones

In [17]:
# Create od friction factor list
od_friction_factor_list = []
for i in range(len(g_o_zone_id_list)):
    od_distance = g_od_distance_list[i]
    travel_time = (max(od_distance, 0.001) / 1000) / average_travel_speed * 60  # unit: min
    friction_factor = a * (travel_time ** b) * (np.exp(c * travel_time))

    od_friction_factor_list.append(friction_factor) if od_distance != 0 else od_friction_factor_list.append(0)

# Create od friction facotr matrix
number_of_zones = len(g_zone_list)
friction_factor_matrix = np.zeros((number_of_zones, number_of_zones))
for i in range(len(od_friction_factor_list)):
    o_zone_id = g_o_zone_id_list[i]
    o_zone_index = g_zone_id_to_index_dict[o_zone_id]
    d_zone_id = g_d_zone_id_list[i]
    d_zone_index = g_zone_id_to_index_dict[d_zone_id]
    friction_factor_matrix[o_zone_index][d_zone_index] = od_friction_factor_list[i]
# print(friction_factor_matrix)

KeyError: 6

## 2.3 Calculate total attraction friction for each origin zone

In [64]:
# Generate attraction friction
len_zone_list = len(g_zone_list)
total_attraction_friction_list = np.zeros(len_zone_list)
for i in range(len_zone_list):
    prod_zone_id = g_zone_list[i].id
    prod_zone_index = g_zone_id_to_index_dict[prod_zone_id]
    for j in range(len_zone_list):
        attr_zone_id = g_zone_list[j].id
        attr_zone_index = g_zone_id_to_index_dict[attr_zone_id]
        total_attraction_friction_list[prod_zone_index] += g_zone_list[attr_zone_index].attraction * \
                                                           friction_factor_matrix[prod_zone_index][attr_zone_index]


## 2.4 Calculate OD volume between zones

In [68]:
# Generate OD volume 
od_volume_list = []
for i in range(len(g_o_zone_id_list)):
    prod_zone_id = g_o_zone_id_list[i]
    prod_zone_index = g_zone_id_to_index_dict[prod_zone_id]
    attr_zone_id = g_d_zone_id_list[i]
    attr_zone_index = g_zone_id_to_index_dict[attr_zone_id]
    od_volume = g_zone_list[prod_zone_index].production * g_zone_list[attr_zone_index].attraction * od_friction_factor_list[i] \
                / max(0.000001, total_attraction_friction_list[prod_zone_index])
    od_volume_list.append(round(od_volume))
print('total travel demand:',sum(od_volume_list))

total travel demand: 9955


## 2.5 Update OD information

In [69]:
# Update OD
print('\nTop 10 O/D Volume:')
volume_idx=sorted(enumerate(od_volume_list), key=lambda od_id:od_id[1], reverse = True)
for od in range(10):
    index_truple = volume_idx[od]
    print(f'Top {str(od + 1)} O/D pair: zone {str(g_o_zone_id_list[index_truple[0]])} ->zone {str(g_d_zone_id_list[index_truple[0]])}, volume = {str(index_truple[1])}')

data_demand = pd.DataFrame({
    "o_zone_id": g_o_zone_id_list,
    "o_zone_name": g_o_zone_name_list,
    "d_zone_id": g_d_zone_id_list,
    "d_zone_name": g_d_zone_name_list,
    "accessibility": g_od_distance_list,
    "volume": od_volume_list,
    "geometry": g_od_geometry_list
})

print('\n',data_demand)


Top 10 O/D Volume:
Top 1 O/D pair: zone 13->zone 7, volume = 89
Top 2 O/D pair: zone 13->zone 14, volume = 75
Top 3 O/D pair: zone 10->zone 7, volume = 71
Top 4 O/D pair: zone 7->zone 13, volume = 70
Top 5 O/D pair: zone 10->zone 13, volume = 65
Top 6 O/D pair: zone 13->zone 8, volume = 65
Top 7 O/D pair: zone 1->zone 24, volume = 62
Top 8 O/D pair: zone 13->zone 10, volume = 57
Top 9 O/D pair: zone 14->zone 13, volume = 56
Top 10 O/D pair: zone 11->zone 13, volume = 53

       o_zone_id o_zone_name  d_zone_id d_zone_name  accessibility  volume  \
0             1                      2                    2512.71      31   
1             1                      3                    2993.49      36   
2             1                      4                    4695.65      11   
3             1                      5                   10956.11       3   
4             1                      6                     480.49      46   
...         ...         ...        ...         ...          

## 2.6 Save to demand.csv

In [70]:
#  Use absolute path or relative path to save the file
path_demand = "./demand.csv"
data.to_csv('demand.csv', index=False, line_terminator='\n')

# Final Code


In [13]:
import os
import csv
import numpy as np
import pandas as pd 
from math import acos, sin, cos


class Zone:
    def __init__(self):
        self.id = 0
        self.name = ''  # alphanumeric name of this zone, such as A1, A2,...
        self.bin_index = 0
        self.activity_nodes = ''  # activity nodes which belong to this zone

        self.centroid = ''  # centroid coordinate (x, y) based on wkt format
        self.centroid_x = ''
        self.centroid_y = ''
        self.geometry = ''  # zone boundary based on wkt format

        self.production = 0.0
        self.attraction = 0.0

def read_zone_csv(path_zone_csv: str = None) -> list:
    """read zone.csv obtained from path4gmns"""

    number_of_zones = 0
    g_zone_list = []
    g_zone_id_to_index_dict = {}

    with open(path_zone_csv, errors='ignore') as fp:
        reader = csv.DictReader(fp)

        print("Start reading zone.csv")
        zone_index = 0
        for line in reader:

            # Create Zone objects
            if line.get('zone_id', None):

                # initialize a zone object
                zone = Zone()

                zone.id = int(line.get("zone_id")) if line.get("zone_id") else 0
                g_zone_id_to_index_dict[zone.id] = zone_index if line.get('zone_id') else None

                zone.bin_index = int(line.get('bin_index')) if line.get('bin_index') else 0
                zone.activity_nodes = line.get('activity_nodes') or ''

                zone.centroid_x = line.get('x_coord') or ''
                zone.centroid_y = line.get('y_coord') or ''
                zone.centroid = f'POINT ({zone.centroid_x} {zone.centroid_y})'
                zone.geometry = line.get('geometry') or ''

                zone.production = float(line.get('production')) if line.get('production') else 0.0
                zone.attraction = float(line.get('attraction')) if line.get('attraction') else 0.0

                zone_index += 1
                number_of_zones += 1
                g_zone_list.append(zone)

    if number_of_zones == 0:
        print("zone information is empty in zone.csv! Please check it.")
    else:
        print("Finished! The number of zones is", number_of_zones)

    return [g_zone_list, g_zone_id_to_index_dict]

def convert_geometry_str_to_points(geometry_str: str) -> list:
    """Convert geometry string to a list of points
    Args:
        geometry_str (str): LINESTRING (-122.28335 47.6470736,-122.28335 47.6470736)
    Return:
        point1 (list): [longitude, latitude]
		point2 (list): [longitude, latitude]
    """

    point1, point2 = [i.split(" ") for i in geometry_str.split(
        "LINESTRING ")[-1][1:-1].split(",")]
    return [point1, point2]

def calculate_distance_from_two_points(point1: list, point2: list) -> float:
	"""Calculate distance between two points on the earth surface based on longitude and latitude

	Args:
		point1 (list): [longitude, latitude] of point 1
		point2 (list): [longitude, latitude] of point 2

 	Return:
  		distance (float): unit in mile
	"""

	Equatorial_Radius = 3963.19059 # unit: mile

	p1lng = np.radians(float(point1[0]))
	p1lat = np.radians(float(point1[1]))
	p2lng = np.radians(float(point2[0]))
	p2lat = np.radians(float(point2[1]))

	distance=acos(sin(p1lat)*sin(p2lat)+cos(p1lat)*cos(p2lat)*cos(p2lng-p1lng)) * Equatorial_Radius
	return distance

def read_accessibility_csv(path_accessibility: str) -> list:
    """ Read od_accessibility.csv obtained from path4gmns"""

    # initialize variables
    g_o_zone_id_list = []
    g_o_zone_name_list = []
    g_d_zone_id_list = []
    g_d_zone_name_list = []
    g_od_distance_list = []
    g_od_geometry_list = []

    # read file and store data
    with open(path_accessibility, errors='ignore') as fp:
        reader = csv.DictReader(fp)

        print("\nStart reading od_accessibility.csv")
        for line in reader:

            # origin and destination zone ids are valid
            if all([line.get('o_zone_id'), line.get('d_zone_id')]):

                g_o_zone_id_list.append(int(line.get('o_zone_id')))
                g_o_zone_name_list.append(line.get('o_zone_name'))

                g_d_zone_id_list.append(int(line.get('d_zone_id')))
                g_d_zone_name_list.append(line.get('d_zone_name'))

                geometry_value = line.get('geometry') or ''
                g_od_geometry_list.append(geometry_value)

                try:
                    # if distance column exists
                    g_od_distance_list.append(float(line['distance']))
                except Exception:
                    # geometry value is not None or ''
                    if geometry_value:
                        point1, point2 = convert_geometry_str_to_points(
                            geometry_value)
                        g_od_distance_list.append(
                            calculate_distance_from_two_points(point1, point2))
                    else:
                        g_od_distance_list.append(float(999999))
                        print(
                            f"distance between OD pair {line['o_zone_id']}-{line['d_zone_id']} is None in od_accessibility.csv! Default distance 999999 is used.")
                        print(
                            f"geometry field is empty for OD pair {line['o_zone_id']}-{line['d_zone_id']} in od_accessibility.csv! Please check it.")

    nearest_distance = min(g_od_distance_list)
    farthest_distance = max(g_od_distance_list)
    print("Finished!")
    print("The nearest distance between OD pairs is", nearest_distance)
    print("The farthest distance between OD pairs is", farthest_distance)

    return [g_o_zone_id_list, g_d_zone_id_list, g_o_zone_name_list, g_d_zone_name_list, g_od_distance_list, g_od_geometry_list]

def traffic_demand_analysis(path_zone_csv: str, path_accessibility: str, path_demand_output: str) -> pd.DataFrame:

    g_zone_list, g_zone_id_to_index_dict = read_zone_csv(path_zone_csv)
    g_o_zone_id_list,g_d_zone_id_list,g_o_zone_name_list,g_d_zone_name_list,g_od_distance_list,g_od_geometry_list = read_accessibility_csv(path_accessibility)

    # Generate gamma function coefficients for friction factors
    trip_purpose_list = ['HBW', 'HBO', 'NHB']
    a_list = [28507, 139173, 219113]
    b_list = [-0.02, -1.285, -1.332]
    c_list = [-0.123, -0.094, -0.1]

    data = pd.DataFrame({
        'trip_purpose': trip_purpose_list,
        'a': a_list,
        'b': b_list,
        'c': c_list})

    # Calculate travel time between zones (unit: km/h)
    average_travel_speed = 30
    trip_purpose_index = 0
    a, b, c = [factor[trip_purpose_index] for factor in [a_list, b_list, c_list]]

    # Create od friction factor list
    od_friction_factor_list = []
    for i in range(len(g_o_zone_id_list)):
        od_distance = g_od_distance_list[i]
        travel_time = (max(od_distance, 0.001) / 1000) / average_travel_speed*60  # unit: min
        friction_factor = a * (travel_time ** b) * (np.exp(c * travel_time))

        od_friction_factor_list.append(
            friction_factor) if od_distance != 0 else od_friction_factor_list.append(0)

    # Create od friction facotr matrix
    number_of_zones = len(g_zone_list)
    friction_factor_matrix = np.zeros((number_of_zones, number_of_zones))
    for i in range(len(od_friction_factor_list)):
        o_zone_id = g_o_zone_id_list[i]
        o_zone_index = g_zone_id_to_index_dict[o_zone_id]
        d_zone_id = g_d_zone_id_list[i]
        d_zone_index = g_zone_id_to_index_dict[d_zone_id]
        friction_factor_matrix[o_zone_index][d_zone_index] = od_friction_factor_list[i]
    # print(friction_factor_matrix)

    # Generate attraction friction
    len_zone_list = len(g_zone_list)
    total_attraction_friction_list = np.zeros(len_zone_list)
    for i in range(len_zone_list):
        prod_zone_id = g_zone_list[i].id
        prod_zone_index = g_zone_id_to_index_dict[prod_zone_id]
        for j in range(len_zone_list):
            attr_zone_id = g_zone_list[j].id
            attr_zone_index = g_zone_id_to_index_dict[attr_zone_id]
            total_attraction_friction_list[prod_zone_index] += g_zone_list[attr_zone_index].attraction * \
                friction_factor_matrix[prod_zone_index][attr_zone_index]

    # Generate OD volume
    od_volume_list = []
    for i in range(len(g_o_zone_id_list)):
        prod_zone_id = g_o_zone_id_list[i]
        prod_zone_index = g_zone_id_to_index_dict[prod_zone_id]
        attr_zone_id = g_d_zone_id_list[i]
        attr_zone_index = g_zone_id_to_index_dict[attr_zone_id]
        od_volume = g_zone_list[prod_zone_index].production * g_zone_list[attr_zone_index].attraction * od_friction_factor_list[i] \
            / max(0.000001, total_attraction_friction_list[prod_zone_index])
        od_volume_list.append(round(od_volume))
    print('total travel demand:', sum(od_volume_list))

    # Update OD
    print('\nTop 10 O/D Volume:')
    volume_idx = sorted(enumerate(od_volume_list), key=lambda od_id: od_id[1], reverse=True)
    for od in range(10):
        index_truple = volume_idx[od]
        print(
            f'Top {str(od + 1)} O/D pair: zone {str(g_o_zone_id_list[index_truple[0]])} ->zone {str(g_d_zone_id_list[index_truple[0]])}, volume = {str(index_truple[1])}')

    data_demand = pd.DataFrame({
        "o_zone_id": g_o_zone_id_list,
        "o_zone_name": g_o_zone_name_list,
        "d_zone_id": g_d_zone_id_list,
        "d_zone_name": g_d_zone_name_list,
        "accessibility": g_od_distance_list,
        "volume": od_volume_list,
        "geometry": g_od_geometry_list
    })

    print('\n', data_demand)

    #  Use absolute path or relative path to save the file
    data_demand.to_csv(path_demand_output, index=False, line_terminator='\n')
    return data_demand


if __name__ == "__main__":

    path_zone_csv = "C:\\Users\\18016\\zone.csv"
    path_od_accessibility  = "C:\\Users\\18016\\od_accessibility.csv"
    path_demand_output = "C:\\Users\\18016\\demand.csv"

    data_demand = traffic_demand_analysis(path_zone_csv, path_od_accessibility, path_demand_output)




Start reading zone.csv
Finished! The number of zones is 89

Start reading od_accessibility.csv
Finished!
The nearest distance between OD pairs is 0.06
The farthest distance between OD pairs is 4.76
total travel demand: 27335

Top 10 O/D Volume:
Top 1 O/D pair: zone 100003 ->zone 100008, volume = 253
Top 2 O/D pair: zone 100003 ->zone 100005, volume = 239
Top 3 O/D pair: zone 100003 ->zone 100006, volume = 235
Top 4 O/D pair: zone 100005 ->zone 100003, volume = 233
Top 5 O/D pair: zone 100003 ->zone 100011, volume = 224
Top 6 O/D pair: zone 100003 ->zone 100009, volume = 219
Top 7 O/D pair: zone 100008 ->zone 100003, volume = 177
Top 8 O/D pair: zone 100005 ->zone 100008, volume = 174
Top 9 O/D pair: zone 100014 ->zone 100003, volume = 170
Top 10 O/D pair: zone 100006 ->zone 100003, volume = 165

       o_zone_id o_zone_name  d_zone_id d_zone_name  accessibility  volume  \
0        100001        None     100002        None           1.27      32   
1        100001        None     100003